In [37]:
import json
import openpyxl
import re

# -------------------- 標題清洗pattern -----------------------
# 取代為""   【xxx】：
pattern_title = re.compile(r'【快訊】|【好文回顧】|【更新】|【影片】|【端傳媒】|【不斷更新】|【美麗佳人】|【持續更新】|【獨家】');

# -------------------- 內文清洗pattern -----------------------

pattern_conn = re.compile(r'(http://|https://|www)[a-zA-Z0-9\\./_]+')

pattern_flower = re.compile(r'(✿|❤|\(\)|（）|　|◆|＊|▼|●|★|☆|♥|♫|①|②|③|④|⑤|⑥|⑦|⑧|⑨|⑩|►|⬇|➡|▲)|※|＃|▶|○')
pattern_flower1 = re.compile(r'\(\*´•ω•`\*\)/♡\*|～\^\^～\)|ლ\(･\(エ\)･|\(๑´ㅂ`๑\)|！\(=￣ω￣=\)|（◎`·ω·\'）人（\'·ω·`\*）')

pattern_1 = re.compile(
    r'（圖／翻攝.{0,10}|（@.{0,10}）|（新增網友留言）|(首圖.{0,15})|（新增影片）|（圖片取自.{0,15}臉書）|（以上圖片皆為示意圖）|（取.{0,10}粉絲團）|（點超連結看詳細介紹）|（更新影音新聞、網友回應）|（影片.{0,2}/.{0,50}）|攝:AFP/GoogleDeepMind/GOOGLE|（.{1,30}／.{1,30}）|（.{1,30}/.{1,30}）|\(.{1,30}/.{1,30}\)|（圖.{0,30}）|（影片.{0,30}）|（示意.{0,20}）|（東森新聞資料畫面）|（新增.{0,10}來源）|【.{0,20}】')
pattern_3 = re.compile(
    r'圖/.{1,15}臉書|圖/唐立淇占星幫|圖/.{0,4}社|圖／.{0,20}推特|圖/.{0,10}影片/|圖/DailyView網路溫度計|圖/|圖／.{1,15}臉書|圖／|圖/.{0,5}微博|文／')
pattern_2 = re.compile(
    r'.{0,20}Facebook）||微博）|/東森新聞\)|.{0,11}臉書.{0,10}）|facebook.{0,30}\)|twitter）|Instagram|推特）|/取自鉅亨網\)|文章來源：.{0,6}|OnLine）|Online）|moviepicks】.{0,15}提供）|Relief）|YouTube）|Youtube）|IG）|map）|Map）|News）|Salads）|DuckCompany）|Twitter）|攝）|.{0,15}併圖）|臉書、pixabay）|news）|熱講香港、東森新聞）|臉書社團《爆廢公社》）|示意圖.{0,10}）|YOUTUBE）')

pattern_4 = re.compile(r'Source from《.{1,100}》|鏡週刊關心您：未滿18歲禁止飲酒，飲酒過量害人害己，酒後不開車，安全有保障。珍惜生命拒絕毒品，健康無價不容毒噬。遠離家庭暴力，可通報全國保護專線113鏡週刊關心您，再給自己一次機會自殺諮詢專線：0800-788995（24小時）生命線：1995張老師專線：1980揍子他哭求鬧自殺的媽：沒錢幫妳辦後事由《鏡週刊》授權提供。')
pattern_5 = re.compile(r'想爆料、投訴、分享自己的生活點滴嗎？快來東森新聞「爆一爆」|newsreader.{0,10}tw|馬上按讚 加入噪咖粉絲團') # !!!

pattern_6 = re.compile(r'想看更多.{1,10}的作品嗎，請上【YouTube頻道】，或是到粉絲團追蹤「.{1,10}」，精彩內容不錯過喔！|觀看影片請點此')


pattern_7 = re.compile(r'東森新聞關心您.{0,150}119|東森新聞關心您{0,100}1980|影片取自YouTube，如遭刪除請見諒。|若影片無法觀看請|尊重身體自主權！請撥打113、110')
pattern_8 = re.compile(r'生命誠可貴.{1,150}1980|家暴求助專線.{1,40}119|服務時間：週一至週五 9：00～18：00|自殺防治諮詢.{1,50}0')
pattern_9 = re.compile(r'本篇文章由.{0,10}授權提供※.{0,10}官網※.{0,10}部落格|示意圖，非當事畫面。|本文獲授權，請勿任意轉載！|民俗說法，僅供參考')
pattern_10 = re.compile(r'東森電視版權所有.{1,30}。|本文.{1,20}提供，未經授權，請勿轉載！|Claire美麗佳人提供，未經授權，請勿轉載！】|作者言論不代表東森立場授權轉載。')
# -------------------- pattern宣告結束 -----------------------


workbook = openpyxl.Workbook()
sheet = workbook.worksheets[0]
listtitle = ["title", "date", "author", "text", "url", "tags", "type_list", "source", "views", "share", "like"]
sheet.append(listtitle)
ILLEGAL_CHARACTERS_RE = re.compile(r'[\000-\010]|[\013-\014]|[\016-\037]')
ILLEGAL_CHARACTERS_EMOJI = re.compile(u'[\U00010000-\U0010ffff]')
with open("C:/Users/Student/Desktop/ebc/20170101_0430.json", "r", encoding='utf8') as jsonfile:
    data = json.load(jsonfile) 
    for i in data.keys():
        if not data[i]["text"]:
            continue
        
        tags = ""
        if data[i]['tags']:
            for tagstring in data[i]['tags']:
                tags = tags + tagstring + "、"
            tags = tags[0:-1]
            # print(tags)

        type_list = ""
        if data[i]['type_list']:
            for type_liststring in data[i]['type_list']:
                type_list = type_list + type_liststring + "、"
            type_list = type_list[0:-1]
             # print(type_list)
       
     
        # -------------------- 標題清洗 -----------------------
        title = data[i]['title'];
        if re.search(pattern_title, title):
            title = pattern_title.sub('', title);

        # -------------------- 內文清洗 -----------------------
        content= data[i]['text']
        check_sup = content.__contains__('【今日最熱門】');
        if check_sup:
            split_sup = content.split('【今日最熱門】');
            content = split_sup[0];


        # -------------------- 取代單個 -----------------------
        check_intern = re.search(pattern_1, content);
        if check_intern:
            content = pattern_1.sub('', content);

        check_fb = re.search(pattern_2, content);
        if check_fb:
            content = pattern_2.sub('', content);

        check_sentence = re.search(pattern_3, content);
        if check_sentence:
            content = pattern_3.sub('', content);

        check_report = re.search(pattern_4, content);
        if check_report:
            content = pattern_4.sub('', content);

        check_fb = re.search(pattern_5, content);
        if check_fb:
            content = pattern_5.sub('', content);

        check_fb = re.search(pattern_6, content);
        if check_fb:
            content = pattern_6.sub('', content);

        check_fb = re.search(pattern_7, content);
        if check_fb:
            content = pattern_7.sub('', content);

        check_fb = re.search(pattern_8, content);
        if check_fb:
            content = pattern_8.sub('', content);
        check_fb = re.search(pattern_9, content);
        if check_fb:
            content = pattern_9.sub('', content);
        check_fb = re.search(pattern_10, content);
        if check_fb:
            content = pattern_10.sub('', content);

        check_flower = re.search(pattern_flower, content);
        if check_flower:
            content = pattern_flower.sub('', content);

        check_conn = re.search(pattern_conn, content);
        if check_conn:
            content = pattern_conn.sub('', content);
        # -------------------- 清洗結束 -----------------------
        row = [title, data[i]['time'].split('_')[0], data[i]['author'], content,
               data[i]['url'] + " ", tags, type_list, data[i]['source'], data[i]['like'], data[i]['share'],
               data[i]['views']];
        count = 0
        for i in row:
            row[count] = ILLEGAL_CHARACTERS_RE.sub(r'', row[count])
            row[count] = ILLEGAL_CHARACTERS_EMOJI.sub(r'', row[count])
            count += 1
        sheet.append(row)
    
        
workbook.save("C:/Users/Student/Desktop/201701010430.xlsx")


In [19]:
workbook = openpyxl.Workbook()
sheet = workbook.worksheets[0]
listtitle=["title","date","author","text","url","tags", "type_list","source","views","share","like"]
sheet.append(listtitle)
ILLEGAL_CHARACTERS_RE = re.compile(r'[\000-\010]|[\013-\014]|[\016-\037]')
with open ("C:/Users/Student/Desktop/ebc/20190301_0415.json","r",encoding='utf8') as jsonfile:
    data = json.load(jsonfile)
    for i in data.keys():
        
        tags = ""
        if data[i]['tags']:
            for tagstring in data[i]['tags']:
                tags = tags + tagstring + "、"
            tags = tags[0:-1]
            # print(tags)

        type_list = ""
        if data[i]['type_list']:
            for type_liststring in data[i]['type_list']:
                type_list = type_list + type_liststring + "、"
            type_list = type_list[0:-1]
            # print(type_list)

        row = [data[i]['title'], data[i]['time'].split('_')[0], data[i]['author'], data[i]['text'], data[i]['url']+" ", tags, type_list, data[i]['source'], data[i]['like'], data[i]['share'], data[i]['views']]
        count = 0
        for i in row:
            row[count] = ILLEGAL_CHARACTERS_RE.sub(r'', i)
            count += 1
        sheet.append(row)
workbook.save("C:/Users/Student/Desktop/ebc_rowdata/20190301_0415.xlsx")

In [3]:
import requests
from bs4 import BeautifulSoup
import re
import  datetime
import json

##生成日期列表
def generateDatelist(dateForm = '%Y%m%d',start=datetime.datetime.now().strftime('%Y%m%d'),end=datetime.datetime.now().strftime('%Y%m%d')):
    datelist = []
    datelist.append(start)
    start = datetime.datetime.strptime(start,dateForm)
    end = datetime.datetime.strptime(end,dateForm)
    while start < end:
        start = start+datetime.timedelta(days=+1)
        start = start.strftime(dateForm)
        datelist.append(start)
        start = datetime.datetime.strptime(start,dateForm)
    return generateURL(datelist)

##將日期轉成所需的url
def generateURL(datelist):
    url_list = ["https://news.ebc.net.tw/Search/Result?type=date&value=" + date[0:4] +"%252F"+date[4:6]+"%252F"+date[-2:] for date in datelist]
    return  url_list


#獲得日期列表

dayStart ="20181001"
dayEnd = "20181231"
dateForm = '%Y%m%d'

content ={}


url_list = generateDatelist(dateForm,dayStart,dayEnd)
for url in url_list:
    print(url)

    for round in range(6):
        r = requests.get(url)
        soup = BeautifulSoup(r.text, 'html.parser')
        r.encoding = 'utf-8'
        # print(res.text)

        paging = soup.select('div.page-area.white-box div a')
        if paging[-1].string != '＞' :
            url = url

        else :
            next_url = 'https://news.ebc.net.tw' + paging[-1]['href']
            url = next_url

        target_list = soup.find_all('div', class_='news-list-box')

        for i in target_list:
            news_list = i.find_all('a')
            time_list = i.find_all('span', class_='small-gray-text')

            for i in news_list:
                if i.attrs.get('href') != None:
                    new_url = 'https://news.ebc.net.tw' + i.attrs.get('href')
                    print(new_url)
                    res = requests.get(new_url)
                    soup = BeautifulSoup(res.text, 'html.parser')
                    r.encoding = 'utf-8'
                    # TAGS
                    taglist = soup.select('div.keyword a')
                    tags = []
                    if len(taglist) >= 1:
                        for i in taglist:
                            tags.append(i.text)
                        # print(tags)
                    # print(new_url)

                    # TITLE
                    title = soup.find('h1').text
                    # print('title:' + title)
                    # TIME
                    time_auth = soup.select('div.info span.small-gray-text')[0].string.strip().split()
                    time = time_auth[0] + "_" + time_auth[1]
                    if len(time_auth) > 3:
                        author = time_auth[-1]
                    elif len(time_auth) == 3:

                        if time_auth[-1][0:2] == "東森":
                            author = ""
                        elif time_auth[-1][0:3] == "端傳媒":
                            author = time_auth[-1][-3:]
                        else:
                            author = time_auth[-1]
                    else:
                        author = ""

                    # time  = append(time1)
                    # print('time:' + time)
                    # print('author:' + author)

                    # LIST
                    type_list = []
                    group = soup.find(id='web-map')
                    type = group.find_all('a')[1].text
                    type_list.append(type)
                    # print('type_list:' + type)

                    # TEXT
                    txt_div = soup.find('div', {'class': 'raw-style'})
                    txt_target = txt_div.select('span',style='data-reactroot p')

                    pre_text = txt_div.get_text()
                    text = ""
                    for i in pre_text.split():
                        if i[0] != "▼":
                            if i[0] != "▲":
                                if i[0] != "★":
                                     if i[0] != "圖":
                                        text += i

                    c1 = text.split()
                    text = ''.join(c1)
                    # print('text:' + text)
                    # print(" ")

                    # 塞進字典
                    # content = {};
                    dic = {};
                    dic['title'] = title;
                    dic['time'] = time;
                    dic['author'] = author;
                    dic['text'] = text;
                    dic['url'] = new_url;
                    dic['tags'] = tags;
                    dic['type_list'] = type_list;
                    dic['source'] = '東森';
                    dic['views'] = '';
                    dic['share'] = '';
                    dic['like'] = ''
                    if author:
                        conKey = time + '_' + author
                        content[conKey] = dic

                    else:
                        conKey = time + '_' + "無名氏"
                        content[conKey] = dic
                    # print(content)

            with open("C:/Users/Student/Desktop/20181001_1231.json", "w") as fjson:
                json.dump(content, fjson)
                print('.')


https://news.ebc.net.tw/Search/Result?type=date&value=2018%252F10%252F01
https://news.ebc.net.tw/News/Article/132890
https://news.ebc.net.tw/News/Article/132889
https://news.ebc.net.tw/News/Article/132888
https://news.ebc.net.tw/News/Article/132887
https://news.ebc.net.tw/News/Article/132886
https://news.ebc.net.tw/News/Article/132885
https://news.ebc.net.tw/News/Article/132884
https://news.ebc.net.tw/News/Article/132883
https://news.ebc.net.tw/News/Article/132882
https://news.ebc.net.tw/News/Article/132881
https://news.ebc.net.tw/News/Article/132880
https://news.ebc.net.tw/News/Article/132879
https://news.ebc.net.tw/News/Article/132878
https://news.ebc.net.tw/News/Article/132877
https://news.ebc.net.tw/News/Article/132875
https://news.ebc.net.tw/News/Article/132876
https://news.ebc.net.tw/News/Article/132874
https://news.ebc.net.tw/News/Article/132873
https://news.ebc.net.tw/News/Article/132872
https://news.ebc.net.tw/News/Article/132871
https://news.ebc.net.tw/News/Article/132870
htt

https://news.ebc.net.tw/News/Article/133010
https://news.ebc.net.tw/News/Article/133011
https://news.ebc.net.tw/News/Article/133009
https://news.ebc.net.tw/News/Article/133008
https://news.ebc.net.tw/News/Article/133005
https://news.ebc.net.tw/News/Article/133006
https://news.ebc.net.tw/News/Article/133003
https://news.ebc.net.tw/News/Article/133004
https://news.ebc.net.tw/News/Article/133007
https://news.ebc.net.tw/News/Article/133001
https://news.ebc.net.tw/News/Article/133000
https://news.ebc.net.tw/News/Article/132998
https://news.ebc.net.tw/News/Article/132997
https://news.ebc.net.tw/News/Article/132996
https://news.ebc.net.tw/News/Article/132994
https://news.ebc.net.tw/News/Article/132999
https://news.ebc.net.tw/News/Article/132995
https://news.ebc.net.tw/News/Article/132993
https://news.ebc.net.tw/News/Article/132990
https://news.ebc.net.tw/News/Article/132991
https://news.ebc.net.tw/News/Article/132992
https://news.ebc.net.tw/News/Article/132989
https://news.ebc.net.tw/News/Art

https://news.ebc.net.tw/News/Article/133145
https://news.ebc.net.tw/News/Article/133144
https://news.ebc.net.tw/News/Article/133143
https://news.ebc.net.tw/News/Article/133141
https://news.ebc.net.tw/News/Article/133142
https://news.ebc.net.tw/News/Article/133140
https://news.ebc.net.tw/News/Article/133139
https://news.ebc.net.tw/News/Article/133138
https://news.ebc.net.tw/News/Article/133136
https://news.ebc.net.tw/News/Article/133135
https://news.ebc.net.tw/News/Article/133134
https://news.ebc.net.tw/News/Article/133133
https://news.ebc.net.tw/News/Article/133137
https://news.ebc.net.tw/News/Article/134024
https://news.ebc.net.tw/News/Article/133132
https://news.ebc.net.tw/News/Article/133130
.
https://news.ebc.net.tw/News/Article/133131
https://news.ebc.net.tw/News/Article/133129
https://news.ebc.net.tw/News/Article/133127
https://news.ebc.net.tw/News/Article/133128
https://news.ebc.net.tw/News/Article/133126
https://news.ebc.net.tw/News/Article/133124
https://news.ebc.net.tw/News/A

https://news.ebc.net.tw/News/Article/133245
https://news.ebc.net.tw/News/Article/133247
https://news.ebc.net.tw/News/Article/133244
https://news.ebc.net.tw/News/Article/133242
https://news.ebc.net.tw/News/Article/133243
https://news.ebc.net.tw/News/Article/133240
https://news.ebc.net.tw/News/Article/133241
https://news.ebc.net.tw/News/Article/133239
https://news.ebc.net.tw/News/Article/133237
https://news.ebc.net.tw/News/Article/133238
https://news.ebc.net.tw/News/Article/133248
https://news.ebc.net.tw/News/Article/133235
https://news.ebc.net.tw/News/Article/133232
https://news.ebc.net.tw/News/Article/133231
https://news.ebc.net.tw/News/Article/133230
.
https://news.ebc.net.tw/News/Article/133229
https://news.ebc.net.tw/News/Article/133233
https://news.ebc.net.tw/News/Article/133234
https://news.ebc.net.tw/News/Article/133236
https://news.ebc.net.tw/News/Article/133225
https://news.ebc.net.tw/News/Article/133228
https://news.ebc.net.tw/News/Article/133227
https://news.ebc.net.tw/News/A

https://news.ebc.net.tw/News/Article/133350
https://news.ebc.net.tw/News/Article/133348
https://news.ebc.net.tw/News/Article/133346
https://news.ebc.net.tw/News/Article/133344
.
https://news.ebc.net.tw/News/Article/133345
https://news.ebc.net.tw/News/Article/133343
https://news.ebc.net.tw/News/Article/133341
https://news.ebc.net.tw/News/Article/133347
https://news.ebc.net.tw/News/Article/133339
https://news.ebc.net.tw/News/Article/133338
https://news.ebc.net.tw/News/Article/133337
https://news.ebc.net.tw/News/Article/133336
https://news.ebc.net.tw/News/Article/133335
https://news.ebc.net.tw/News/Article/133334
https://news.ebc.net.tw/News/Article/133333
https://news.ebc.net.tw/News/Article/133331
https://news.ebc.net.tw/News/Article/133332
https://news.ebc.net.tw/News/Article/133304
https://news.ebc.net.tw/News/Article/133330
https://news.ebc.net.tw/News/Article/133340
https://news.ebc.net.tw/News/Article/133329
https://news.ebc.net.tw/News/Article/133328
https://news.ebc.net.tw/News/A

...

https://news.ebc.net.tw/News/Article/145431
https://news.ebc.net.tw/News/Article/145430
https://news.ebc.net.tw/News/Article/145435
https://news.ebc.net.tw/News/Article/145429
https://news.ebc.net.tw/News/Article/145428
https://news.ebc.net.tw/News/Article/145425
https://news.ebc.net.tw/News/Article/145427
https://news.ebc.net.tw/News/Article/145424
https://news.ebc.net.tw/News/Article/145426
https://news.ebc.net.tw/News/Article/145423
https://news.ebc.net.tw/News/Article/145422
https://news.ebc.net.tw/News/Article/145421
https://news.ebc.net.tw/News/Article/145420
https://news.ebc.net.tw/News/Article/145419
https://news.ebc.net.tw/News/Article/145418
https://news.ebc.net.tw/News/Article/145417
.
https://news.ebc.net.tw/Search/Result?type=date&value=2018%252F12%252F27
https://news.ebc.net.tw/News/Article/145756
https://news.ebc.net.tw/News/Article/145755
https://news.ebc.net.tw/News/Article/145754
https://news.ebc.net.tw/News/Article/145752
https://news.ebc.net.tw/News/Article/145751
h

https://news.ebc.net.tw/News/Article/145950
https://news.ebc.net.tw/News/Article/145947
https://news.ebc.net.tw/News/Article/145946
https://news.ebc.net.tw/News/Article/145945
https://news.ebc.net.tw/News/Article/145943
https://news.ebc.net.tw/News/Article/145942
https://news.ebc.net.tw/News/Article/145944
https://news.ebc.net.tw/News/Article/145941
https://news.ebc.net.tw/News/Article/145938
https://news.ebc.net.tw/News/Article/145937
https://news.ebc.net.tw/News/Article/145940
https://news.ebc.net.tw/News/Article/145936
https://news.ebc.net.tw/News/Article/145934
https://news.ebc.net.tw/News/Article/145935
https://news.ebc.net.tw/News/Article/145939
https://news.ebc.net.tw/News/Article/145933
https://news.ebc.net.tw/News/Article/145932
https://news.ebc.net.tw/News/Article/145931
https://news.ebc.net.tw/News/Article/145930
https://news.ebc.net.tw/News/Article/145928
https://news.ebc.net.tw/News/Article/145929
.
https://news.ebc.net.tw/News/Article/145927
https://news.ebc.net.tw/News/A

https://news.ebc.net.tw/News/Article/146081
https://news.ebc.net.tw/News/Article/146080
https://news.ebc.net.tw/News/Article/146079
https://news.ebc.net.tw/News/Article/146007
https://news.ebc.net.tw/News/Article/146078
https://news.ebc.net.tw/News/Article/146076
https://news.ebc.net.tw/News/Article/146001
https://news.ebc.net.tw/News/Article/146075
https://news.ebc.net.tw/News/Article/146074
https://news.ebc.net.tw/News/Article/146073
https://news.ebc.net.tw/News/Article/145989
https://news.ebc.net.tw/News/Article/146071
https://news.ebc.net.tw/News/Article/146072
https://news.ebc.net.tw/News/Article/146070
https://news.ebc.net.tw/News/Article/146069
https://news.ebc.net.tw/News/Article/146068
.
https://news.ebc.net.tw/News/Article/146067
https://news.ebc.net.tw/News/Article/146066
https://news.ebc.net.tw/News/Article/146065
https://news.ebc.net.tw/News/Article/146062
https://news.ebc.net.tw/News/Article/146063
https://news.ebc.net.tw/News/Article/146064
https://news.ebc.net.tw/News/A

https://news.ebc.net.tw/News/Article/146125
https://news.ebc.net.tw/News/Article/146123
https://news.ebc.net.tw/News/Article/146122
https://news.ebc.net.tw/News/Article/146121
https://news.ebc.net.tw/News/Article/146120
https://news.ebc.net.tw/News/Article/146119
https://news.ebc.net.tw/News/Article/146118
.
https://news.ebc.net.tw/News/Article/146117
https://news.ebc.net.tw/News/Article/146116
https://news.ebc.net.tw/News/Article/146114
https://news.ebc.net.tw/News/Article/146115
https://news.ebc.net.tw/News/Article/146113
https://news.ebc.net.tw/News/Article/146111
https://news.ebc.net.tw/News/Article/146112
https://news.ebc.net.tw/News/Article/146110
https://news.ebc.net.tw/News/Article/146109
https://news.ebc.net.tw/News/Article/146108
https://news.ebc.net.tw/News/Article/146107
https://news.ebc.net.tw/News/Article/146106
https://news.ebc.net.tw/News/Article/146105
https://news.ebc.net.tw/News/Article/146104
https://news.ebc.net.tw/News/Article/146102
https://news.ebc.net.tw/News/A

https://news.ebc.net.tw/News/Article/146210
https://news.ebc.net.tw/News/Article/146209
https://news.ebc.net.tw/News/Article/146208
https://news.ebc.net.tw/News/Article/146207
.
https://news.ebc.net.tw/News/Article/146206
https://news.ebc.net.tw/News/Article/146204
https://news.ebc.net.tw/News/Article/146205
https://news.ebc.net.tw/News/Article/146203
https://news.ebc.net.tw/News/Article/146201
https://news.ebc.net.tw/News/Article/146200
https://news.ebc.net.tw/News/Article/146199
https://news.ebc.net.tw/News/Article/146196
https://news.ebc.net.tw/News/Article/146198
https://news.ebc.net.tw/News/Article/146197
https://news.ebc.net.tw/News/Article/146195
https://news.ebc.net.tw/News/Article/146202
https://news.ebc.net.tw/News/Article/146194
https://news.ebc.net.tw/News/Article/146193
https://news.ebc.net.tw/News/Article/146192
https://news.ebc.net.tw/News/Article/146189
https://news.ebc.net.tw/News/Article/146190
https://news.ebc.net.tw/News/Article/146191
https://news.ebc.net.tw/News/A